In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### KAIST Scene Text Database 전처리기:
1. 데이터 url:http://www.iapr-tc11.org/mediawiki/index.php/KAIST_Scene_Text_Database
1. 압축을 다 풀고 'kaist_dataset'아래에 이미지 데이터와 xml 파일을 두면
1. 각 xml 파일에서 글자하나하나 위치정보를 읽어 원본 이미지에서 글자를 별도의 이미지 파일로 저장하는 작업을 수행한다.
1. 결과물은 char_data 아래에 각 글자명으로 폴더를 만들고 그 아래에 원본파일명.글자명.idx.jpg로 저장
    - char_data/{character}/{img_filename}.{character}.{idx}.jpg
    - character가 .(dot)인 경우에는 {character}값을 dot으로 대체하여 처리
    

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import xml.etree.ElementTree

from os import listdir, path, makedirs
from os.path import isfile, isdir, join, basename

img_base_path = 'kaist_dataset'

def plot_img_inline(img):
    plt.figure()
    plt.imshow(np.asarray(img))
    plt.show()

char_count = dict()

def get_unique_img_name(target_path, index_name, char_name):
    idx = 0
    image_filename = '.'.join([index_name, char_name, str(idx), 'jpg'])
    full_path = join(target_path, image_filename)
    
    while path.exists(full_path):
        idx = idx + 1
        image_filename = '.'.join([index_name, char_name, str(idx), 'jpg'])
        full_path = join(target_path, image_filename)
    return full_path

def print_img(img, base_path, index_name, char_name):
    target_path = join(base_path, char_name)
    if not path.exists(target_path):
        makedirs(target_path)

    # 이미지 데이터들이 여러 폴더에 걸쳐있어서 같은 파일명과 같은 글자를 갖는 경우가 많음
    # 뒤에 idx값을 추가해 서로 다른 데이터가 다른 파일명을 갖도록 함.
    full_path = get_unique_img_name(target_path, index_name, char_name)
    img.save(full_path)
    
    if char_name not in char_count:
        char_count[char_name] = 1
    else:
        char_count[char_name] = char_count[char_name] + 1
    
def crop_character_images(base_path, index_name, output_path='char_data'):
    image_filename = '.'.join([index_name, 'jpg'])
    xml_filename = '.'.join([index_name, 'xml'])
    
    # xml 파일에 확장자를 빠트린 경우가 많음. xml 파일이 없는 경우에는 xml파일명을 확장자를 제외한 것으로 설정
    if path.exists(join(base_path, xml_filename)) is False:
        xml_filename = index_name
    im = Image.open(join(base_path, image_filename))
    
#     plot_img_inline(im)
    e = xml.etree.ElementTree.parse(join(base_path, xml_filename)).getroot()

    for image in e:
        for words in image:
            if words.tag == 'words':
                for word in words:
                    for char in word:
                        if len(char.attrib['char']) != 1:
                            continue
                            
                        # 글자가 .인경우 dot으로 대체
                        if char.attrib['char'] == '.':
                            char.attrib['char'] = 'dot'
                            
                        # box 영역으로 crop
                        box = (int(char.attrib['x']), int(char.attrib['y']), int(char.attrib['x']) + int(char.attrib['width']), int(char.attrib['y']) + int(char.attrib['height']))
                        region = im.crop(box)
#                         plot_img_inline(region)

                        # 파일로 저장
                        print_img(region, output_path, index_name, char.attrib['char'])

In [3]:
from os import listdir, path, makedirs
from os.path import isfile, isdir, join, basename

img_base_path = 'kaist_dataset'

fail_filenames = []
def get_recursive_dirlist(base_path):
    # dirname을 얻어옴
    sub_dirnames = [f for f in listdir(base_path) if isdir(join(base_path, f))]
    for dirname in sub_dirnames:
        # 각 dirname별로  recursive 호출
        next_base_path = join(base_path, dirname)
        get_recursive_dirlist(next_base_path)
        
        # 현재 dir에 jpg파일이 있는지 확인
        # KAIST데이터셋은 xxx.jpg, xxx.xml, xxx.bmp 형식으로 있기 때문에, 
        # jpg파일 목록을 가져와서 이미지 데이터 목록을 확보
        data_filenames = [path.splitext(f)[0] for f in listdir(next_base_path) if isfile(join(next_base_path, f)) and f.lower().endswith('.jpg')]        
        for x in data_filenames:
            try:
                crop_character_images(next_base_path, x)
            except:
                # 데이터 형식이 잘못된 경우가 많아 exception이 발생한 경우는 그냥 버림. 대신 뭐가 문제였는지만 출력
                print('###EXCEPTION', next_base_path, x)
                fail_filenames.append((next_base_path, x))

# get_recursive_dirlist(img_base_path)

In [4]:
def read_charlist(base_path):
    char_data = {}
    sub_dirnames = [f for f in listdir(base_path) if isdir(join(base_path, f))]
    for dirname in sub_dirnames:
        # 각 dirname별로  recursive 호출
        next_base_path = join(base_path, dirname)
        
        # 현재 dir에 jpg파일이 있는지 확인
        # KAIST데이터셋은 xxx.jpg, xxx.xml, xxx.bmp 형식으로 있기 때문에, 
        # jpg파일 목록을 가져와서 이미지 데이터 목록을 확보
        data_filenames = [path.splitext(f)[0] for f in listdir(next_base_path) if isfile(join(next_base_path, f)) and f.lower().endswith('.jpg')]        
        
        char_data[dirname] = data_filenames
        
    return char_data
                
char_data = read_charlist('char_data')

dataset = {}
data_charset = {}
for idx, (char, data) in enumerate(char_data.items()):

    if len(data) < 900:
        continue
    print(idx, char, len(data))
    dataset[char] = data
    for x in data:
        data_charset[x] = char


20 8 967
31 E 1185


In [5]:
from functools import reduce
count = reduce(lambda x, y: x + y, [len(x) for k, x in dataset.items()])
print('count data', count, len(data_charset))

count data 2152 2152


In [6]:
data_charset

label_idx = {x:idx for idx, x in enumerate(sorted(list(set(data_charset.values()))))}

label_idx

data_idxcharset = {fname:label_idx[char] for fname, char in data_charset.items()}
data_idxcharset

{'002.8.0': 0,
 '006.8.0': 0,
 '006.8.1': 0,
 '008.8.0': 0,
 '009.8.0': 0,
 '009.8.1': 0,
 '011.8.0': 0,
 '013.8.0': 0,
 '013.8.1': 0,
 '014.8.0': 0,
 '017.8.0': 0,
 '023.8.0': 0,
 '025.8.0': 0,
 '025.8.1': 0,
 '025.8.2': 0,
 '025.8.3': 0,
 '025.8.4': 0,
 '025.8.5': 0,
 '027.8.0': 0,
 '027.8.1': 0,
 '028.8.0': 0,
 '029.8.0': 0,
 '032.8.0': 0,
 '032.8.1': 0,
 '032.8.2': 0,
 '035.8.0': 0,
 '035.8.1': 0,
 '040.8.0': 0,
 '042.8.0': 0,
 '047.8.0': 0,
 '047.8.1': 0,
 '047.8.2': 0,
 '048.8.0': 0,
 '052.8.0': 0,
 '054.8.0': 0,
 '054.8.1': 0,
 '054.8.2': 0,
 '056.8.0': 0,
 '059.8.0': 0,
 '069.8.0': 0,
 '070.8.0': 0,
 '071.8.0': 0,
 '077.8.0': 0,
 '080116-0057.8.0': 0,
 '080118-0007.8.0': 0,
 '080119-0018.8.0': 0,
 '080119-0031.8.0': 0,
 '1.8.0': 0,
 '10.8.0': 0,
 '102.8.0': 0,
 '103.8.0': 0,
 '105.8.0': 0,
 '11.8.0': 0,
 '117.8.0': 0,
 '117.8.1': 0,
 '118.8.0': 0,
 '118.8.1': 0,
 '12.8.0': 0,
 '120.8.0': 0,
 '121.8.0': 0,
 '121.8.1': 0,
 '121.8.2': 0,
 '123.8.0': 0,
 '123.8.1': 0,
 '125.8.0': 0

In [7]:
label_idx

{'8': 0, 'E': 1}

In [8]:
data_idxcharset

{'002.8.0': 0,
 '006.8.0': 0,
 '006.8.1': 0,
 '008.8.0': 0,
 '009.8.0': 0,
 '009.8.1': 0,
 '011.8.0': 0,
 '013.8.0': 0,
 '013.8.1': 0,
 '014.8.0': 0,
 '017.8.0': 0,
 '023.8.0': 0,
 '025.8.0': 0,
 '025.8.1': 0,
 '025.8.2': 0,
 '025.8.3': 0,
 '025.8.4': 0,
 '025.8.5': 0,
 '027.8.0': 0,
 '027.8.1': 0,
 '028.8.0': 0,
 '029.8.0': 0,
 '032.8.0': 0,
 '032.8.1': 0,
 '032.8.2': 0,
 '035.8.0': 0,
 '035.8.1': 0,
 '040.8.0': 0,
 '042.8.0': 0,
 '047.8.0': 0,
 '047.8.1': 0,
 '047.8.2': 0,
 '048.8.0': 0,
 '052.8.0': 0,
 '054.8.0': 0,
 '054.8.1': 0,
 '054.8.2': 0,
 '056.8.0': 0,
 '059.8.0': 0,
 '069.8.0': 0,
 '070.8.0': 0,
 '071.8.0': 0,
 '077.8.0': 0,
 '080116-0057.8.0': 0,
 '080118-0007.8.0': 0,
 '080119-0018.8.0': 0,
 '080119-0031.8.0': 0,
 '1.8.0': 0,
 '10.8.0': 0,
 '102.8.0': 0,
 '103.8.0': 0,
 '105.8.0': 0,
 '11.8.0': 0,
 '117.8.0': 0,
 '117.8.1': 0,
 '118.8.0': 0,
 '118.8.1': 0,
 '12.8.0': 0,
 '120.8.0': 0,
 '121.8.0': 0,
 '121.8.1': 0,
 '121.8.2': 0,
 '123.8.0': 0,
 '123.8.1': 0,
 '125.8.0': 0

In [9]:
import tensorflow as tf


with tf.Graph().as_default():
    value = tf.read_file('char_data/1/077.1.1.jpg')
    img = tf.image.decode_jpeg(value)
    sess = tf.Session()
    print(sess.run(img))
    
    

[[[ 45  70 111]
  [ 45  70 111]
  [ 45  70 111]
  [ 45  70 111]
  [ 46  69 111]
  [ 46  69 111]
  [ 46  69 111]
  [ 46  69 111]
  [ 42  63 108]
  [ 51  72 117]
  [ 52  73 120]
  [ 45  66 113]
  [ 46  64 114]
  [ 52  70 120]]

 [[ 45  69 113]
  [ 45  69 113]
  [ 45  69 113]
  [ 45  69 113]
  [ 46  69 113]
  [ 46  69 113]
  [ 46  69 113]
  [ 46  69 113]
  [ 48  69 114]
  [ 47  68 115]
  [ 46  67 114]
  [ 46  67 114]
  [ 51  69 119]
  [ 54  72 122]]

 [[ 45  68 118]
  [ 45  68 118]
  [ 45  69 117]
  [ 45  69 117]
  [ 46  68 117]
  [ 46  68 117]
  [ 46  68 115]
  [ 46  68 115]
  [ 52  73 120]
  [ 46  66 116]
  [ 45  63 113]
  [ 48  66 116]
  [ 48  66 116]
  [ 42  60 110]]

 [[ 45  68 122]
  [ 45  68 122]
  [ 45  68 120]
  [ 45  68 120]
  [ 46  67 120]
  [ 46  68 118]
  [ 46  68 118]
  [ 46  68 117]
  [ 39  59 109]
  [ 43  63 113]
  [ 50  68 118]
  [ 54  72 122]
  [ 55  73 121]
  [ 55  73 121]]

 [[ 46  67 124]
  [ 46  67 124]
  [ 46  67 124]
  [ 46  67 124]
  [ 46  67 122]
  [ 46  67 122]


In [10]:
import numpy as np
label_arr = np.reshape(list(data_idxcharset.values()), (-1, 1))
label_arr

array([[0],
       [0],
       [0],
       ..., 
       [1],
       [1],
       [1]])

In [11]:
import tensorflow as tf
import cifar10


FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('eval_dir', '/tmp/kaist_eval',
                           """Directory where to write event logs.""")
tf.app.flags.DEFINE_string('eval_data', 'test',
                           """Either 'test' or 'train_eval'.""")
tf.app.flags.DEFINE_string('checkpoint_dir', '/tmp/kaist_train_simple',
                           """Directory where to read model checkpoints.""")
tf.app.flags.DEFINE_integer('eval_interval_secs', 60 * 5,
                            """How often to run the eval.""")
tf.app.flags.DEFINE_integer('num_examples', 1000,
                            """Number of examples to run.""")
tf.app.flags.DEFINE_boolean('run_once', False,
                            """Whether to run eval only once.""")


In [12]:
def get_inputs():
    # Create a queue that produces the filenames to read.
    filenames = ['char_data/' + label + '/' + char + '.jpg' for char, label in data_charset.items()]
    filename_queue = tf.train.string_input_producer(filenames)
    image_reader = tf.WholeFileReader()
    # Read a whole file from the queue, the first returned value in the tuple is the
    # filename which we are ignoring.
    _, image_file = image_reader.read(filename_queue)

    # Decode the image as a JPEG file, this will turn it into a Tensor which we can
    # then use in training.
    image_orig = tf.image.decode_jpeg(image_file)
    image = tf.image.resize_images(image_orig, [64, 64])
    image = tf.image.per_image_standardization(image)
    image.set_shape((64, 64, 3))

    tensor_label = tf.constant(list(data_idxcharset.values()), dtype=tf.int64)
    
    label_queue = tf.train.input_producer(tensor_label)
    i_reader = tf.IdentityReader()
    label, v = i_reader.read(label_queue)
    
    print('label_queue', label_queue)
    print('image_file', image_file)
    print('label', label, v)
    
    num_preprocess_threads = 1
    min_queue_examples = 256

    images, label_batch = tf.train.batch(
        [image, label_queue.dequeue()],
        batch_size=FLAGS.batch_size,
        num_threads=num_preprocess_threads,
        capacity=min_queue_examples + 3 * FLAGS.batch_size)

#     tf.summary.image('images', images)
#     tf.summary.histogram('label_batch', label_batch)
#     return images, tf.reshape(label_batch, [FLAGS.batch_size])
    return images, tf.reshape(label_batch, [FLAGS.batch_size])

get_inputs()

label_queue <tensorflow.python.ops.data_flow_ops.FIFOQueue object at 0x112157128>
image_file Tensor("ReaderReadV2:1", shape=(), dtype=string)
label Tensor("ReaderReadV2_1:0", shape=(), dtype=string) Tensor("ReaderReadV2_1:1", shape=(), dtype=string)


(<tf.Tensor 'batch:0' shape=(128, 64, 64, 3) dtype=float32>,
 <tf.Tensor 'Reshape:0' shape=(128,) dtype=int64>)

In [13]:
import re
import time
from datetime import datetime

TOWER_NAME = 'tower'
NUM_CLASSES = len(label_idx)


def _activation_summary(x):
    """Helper to create summaries for activations.

    Creates a summary that provides a histogram of activations.
    Creates a summary that measures the sparsity of activations.

    Args:
      x: Tensor
    Returns:
      nothing
    """
    # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
    # session. This helps the clarity of presentation on tensorboard.
    tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity',
                      tf.nn.zero_fraction(x))


def _variable_on_cpu(name, shape, initializer):
    """Helper to create a Variable stored on CPU memory.

    Args:
      name: name of the variable
      shape: list of ints
      initializer: initializer for Variable

    Returns:
      Variable Tensor
    """
    with tf.device('/cpu:0'):
        dtype = tf.float16 if FLAGS.use_fp16 else tf.float32
        var = tf.get_variable(name, shape, initializer=initializer, dtype=dtype)
    return var


def _variable_with_weight_decay(name, shape, stddev, wd):
    """Helper to create an initialized Variable with weight decay.

    Note that the Variable is initialized with a truncated normal distribution.
    A weight decay is added only if one is specified.

    Args:
      name: name of the variable
      shape: list of ints
      stddev: standard deviation of a truncated Gaussian
      wd: add L2Loss weight decay multiplied by this float. If None, weight
          decay is not added for this Variable.

    Returns:
      Variable Tensor
    """
    dtype = tf.float16 if FLAGS.use_fp16 else tf.float32
    var = _variable_on_cpu(
        name,
        shape,
        tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)
    return var

def inference(images):
    """Build the CIFAR-10 model.

    Args:
      images: Images returned from distorted_inputs() or inputs().

    Returns:
      Logits.
    """
    # We instantiate all variables using tf.get_variable() instead of
    # tf.Variable() in order to share variables across multiple GPU training runs.
    # If we only ran this model on a single GPU, we could simplify this function
    # by replacing all instances of tf.get_variable() with tf.Variable().
    #
    # conv1
    with tf.variable_scope('conv1') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[5, 5, 3, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv1)

    # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                           padding='SAME', name='pool1')
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                      name='norm1')

    # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[5, 5, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv2)

    # norm2
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                      name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1], padding='SAME', name='pool2')

    # local3
    with tf.variable_scope('local3') as scope:
        # Move everything into depth so we can perform a single matrix multiply.
        reshape = tf.reshape(pool2, [FLAGS.batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = _variable_with_weight_decay('weights', shape=[dim, 384],
                                              stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [384], tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
        _activation_summary(local3)

    # local4
    with tf.variable_scope('local4') as scope:
        weights = _variable_with_weight_decay('weights', shape=[384, 192],
                                              stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
        _activation_summary(local4)
    print('local3', local3)
    print('local4', local4)

    # linear layer(WX + b),
    # We don't apply softmax here because
    # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
    # and performs the softmax internally for efficiency.
    with tf.variable_scope('softmax_linear') as scope:
        weights = _variable_with_weight_decay('weights', [192, NUM_CLASSES],
                                              stddev=1 / 192.0, wd=0.0)
        biases = _variable_on_cpu('biases', [NUM_CLASSES],
                                  tf.constant_initializer(0.0))
        
        print('weights', weights)
        print('biases', biases)
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
        
        print('softmax_linear', softmax_linear)
        _activation_summary(softmax_linear)

    return softmax_linear

def _loss(logits, labels):
    """Add L2Loss to all the trainable variables.

    Add summary for "Loss" and "Loss/avg".
    Args:
      logits: Logits from inference().
      labels: Labels from distorted_inputs or inputs(). 1-D tensor
              of shape [batch_size]

    Returns:
      Loss tensor of type float.
    """
    # Calculate the average cross entropy loss across the batch.
    labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels, logits=logits, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)

    # The total loss is defined as the cross entropy loss plus all of the weight
    # decay terms (L2 loss).
    return tf.add_n(tf.get_collection('losses'), name='total_loss')


def train():
    """Train CIFAR-10 for a number of steps."""
    with tf.Graph().as_default():
        images, labels = get_inputs()
        
        print('labels.get_shape()', labels.get_shape())
        global_step = tf.contrib.framework.get_or_create_global_step()
        
        # Build a Graph that computes the logits predictions from the
        # inference model.
        logits = inference(images)
        
        print('images, logits, labels', images, logits, labels)

        # Calculate loss.
        loss = _loss(logits, labels)

        # Build a Graph that trains the model with one batch of examples and
        # updates the model parameters.
        train_op = cifar10.train(loss, global_step)

        class _LoggerHook(tf.train.SessionRunHook):
            """Logs loss and runtime."""

            def begin(self):
                self._step = -1
                self._start_time = time.time()

            def before_run(self, run_context):
                self._step += 1
                return tf.train.SessionRunArgs(loss)  # Asks for loss value.

            def after_run(self, run_context, run_values):
                if self._step % FLAGS.log_frequency == 0:
                    current_time = time.time()
                    duration = current_time - self._start_time
                    self._start_time = current_time

                    loss_value = run_values.results
                    examples_per_sec = FLAGS.log_frequency * FLAGS.batch_size / duration
                    sec_per_batch = float(duration / FLAGS.log_frequency)

                    format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                                  'sec/batch)')
                    print(format_str % (datetime.now(), self._step, loss_value,
                                        examples_per_sec, sec_per_batch))

        with tf.train.MonitoredTrainingSession(
                checkpoint_dir=FLAGS.train_dir,
                hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
                       tf.train.NanTensorHook(loss),
                       _LoggerHook()],
                config=tf.ConfigProto(
                    log_device_placement=FLAGS.log_device_placement)) as mon_sess:
            while not mon_sess.should_stop():
                mon_sess.run(train_op)

In [ ]:
import re
import time
from datetime import datetime
import math

TOWER_NAME = 'tower'
NUM_CLASSES = len(label_idx)


def eval_once(saver, summary_writer, top_k_op, top_5_op, summary_op, logits, labels):
    """Run Eval once.

    Args:
      saver: Saver.
      summary_writer: Summary writer.
      top_k_op: Top K op.
      summary_op: Summary op.
    """
    with tf.Session() as sess:
        ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            # Restores from checkpoint
            saver.restore(sess, ckpt.model_checkpoint_path)
            # Assuming model_checkpoint_path looks something like:
            #   /my-favorite-path/cifar10_train/model.ckpt-0,
            # extract global_step from it.
            global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
        else:
            print('No checkpoint file found')
            return

        # Start the queue runners.
        coord = tf.train.Coordinator()
        try:
            threads = []
            for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
                threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
                                                 start=True))

            num_iter = int(math.ceil(FLAGS.num_examples / FLAGS.batch_size))
            true_count = 0  # Counts the number of correct predictions.
            true_count_top5 = 0  # Counts the number of correct predictions.
            total_sample_count = num_iter * FLAGS.batch_size
            step = 0
            while step < num_iter and not coord.should_stop():
                predictions, predictions_top5, logit_val, label_val = sess.run([top_k_op, top_5_op, logits, labels])
                
                if step % 1000 == 0:
                    print('logit_val, label_val, predictions', logit_val, label_val, predictions)
                true_count += np.sum(predictions)
                true_count_top5 += np.sum(predictions_top5)
                step += 1

            # Compute precision @ 1.
            precision = true_count / total_sample_count
            precision_top5 = true_count_top5 / total_sample_count
            print('%s: precision @ 1 = %.3f' % (datetime.now(), precision))
            print('%s: precision @ 5 = %.3f' % (datetime.now(), precision_top5))

            summary = tf.Summary()
            summary.ParseFromString(sess.run(summary_op))
            summary.value.add(tag='Precision @ 1', simple_value=precision)
            summary.value.add(tag='Precision @ 5', simple_value=precision_top5)
            summary_writer.add_summary(summary, global_step)
        except Exception as e:  # pylint: disable=broad-except
            coord.request_stop(e)

        coord.request_stop()
        coord.join(threads, stop_grace_period_secs=10)


def evaluate():
    """Eval CIFAR-10 for a number of steps."""
    with tf.Graph().as_default() as g:
        # Get images and labels for CIFAR-10.
#         eval_data = FLAGS.eval_data == 'test'
#         images, labels = cifar10.inputs(eval_data=eval_data)
        images, labels = get_inputs()

        # Build a Graph that computes the logits predictions from the
        # inference model.
        logits = inference(images)
        
        # Calculate predictions.
        top_k_op = tf.nn.in_top_k(logits, labels, 1)
        top_5_op = tf.nn.in_top_k(logits, labels, 5)

        # Restore the moving average version of the learned variables for eval.
        variable_averages = tf.train.ExponentialMovingAverage(
            cifar10.MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        # Build the summary operation based on the TF collection of Summaries.
        summary_op = tf.summary.merge_all()
        summary_writer = tf.summary.FileWriter(FLAGS.eval_dir, g)
        
        while True:
            eval_once(saver, summary_writer, top_k_op, top_5_op, summary_op, logits, labels)
            if FLAGS.run_once:
                break
            time.sleep(FLAGS.eval_interval_secs)

evaluate()

label_queue <tensorflow.python.ops.data_flow_ops.FIFOQueue object at 0x112635d30>
image_file Tensor("ReaderReadV2:1", shape=(), dtype=string)
label Tensor("ReaderReadV2_1:0", shape=(), dtype=string) Tensor("ReaderReadV2_1:1", shape=(), dtype=string)
local3 Tensor("local3/local3:0", shape=(128, 384), dtype=float32)
local4 Tensor("local4/local4:0", shape=(128, 192), dtype=float32)
weights <tf.Variable 'softmax_linear/weights:0' shape=(192, 2) dtype=float32_ref>
biases <tf.Variable 'softmax_linear/biases:0' shape=(2,) dtype=float32_ref>
softmax_linear Tensor("softmax_linear/softmax_linear:0", shape=(128, 2), dtype=float32)
INFO:tensorflow:Restoring parameters from /tmp/kaist_train_simple/model.ckpt-1000
logit_val, label_val, predictions [[-0.09229539  0.09464284]
 [-0.10342072  0.10587113]
 [-0.09686636  0.0949266 ]
 [-0.10293135  0.10210144]
 [-0.09943024  0.09701587]
 [-0.09277352  0.10078932]
 [-0.10884885  0.11250196]
 [-0.09705149  0.10466196]
 [-0.10200181  0.10001393]
 [-0.09682138

2017-09-27 13:19:36.588304: precision @ 1 = 0.538
2017-09-27 13:19:36.588410: precision @ 5 = 1.000
INFO:tensorflow:Restoring parameters from /tmp/kaist_train_simple/model.ckpt-1000
logit_val, label_val, predictions [[-0.10441923  0.10663845]
 [-0.09747399  0.10218955]
 [-0.09547443  0.09716506]
 [-0.09789313  0.0961808 ]
 [-0.09335744  0.09952608]
 [-0.1036506   0.10199901]
 [-0.09957126  0.10200364]
 [-0.09479751  0.09255204]
 [-0.09578076  0.09699152]
 [-0.11010598  0.11112876]
 [-0.11299802  0.11249564]
 [-0.10120308  0.10426714]
 [-0.0994664   0.10119577]
 [-0.09338406  0.09932353]
 [-0.09566812  0.09522759]
 [-0.1003835   0.10053156]
 [-0.0951224   0.09727925]
 [-0.09857512  0.09865119]
 [-0.09682218  0.10133385]
 [-0.10086743  0.09979514]
 [-0.09592593  0.09726266]
 [-0.10093042  0.10098937]
 [-0.1059511   0.10524175]
 [-0.10118771  0.09891245]
 [-0.0946753   0.10236295]
 [-0.09506053  0.10062715]
 [-0.09184206  0.09814274]
 [-0.10200181  0.10001393]
 [-0.09812775  0.09341526]
 

2017-09-27 13:29:47.391116: precision @ 1 = 0.540
2017-09-27 13:29:47.391224: precision @ 5 = 1.000
INFO:tensorflow:Restoring parameters from /tmp/kaist_train_simple/model.ckpt-1000
logit_val, label_val, predictions [[-0.10924964  0.10851808]
 [-0.09315252  0.09981143]
 [-0.10167691  0.10723744]
 [-0.09687492  0.09515904]
 [-0.09481782  0.09504896]
 [-0.09374452  0.0979884 ]
 [-0.09620001  0.09467126]
 [-0.09519115  0.09517152]
 [-0.09869616  0.10086637]
 [-0.0963857   0.10214513]
 [-0.10466741  0.11128902]
 [-0.09480216  0.0982912 ]
 [-0.10153206  0.10686427]
 [-0.09699361  0.09539547]
 [-0.09967482  0.10075679]
 [-0.09620271  0.10500164]
 [-0.10033327  0.10648337]
 [-0.09916532  0.10077526]
 [-0.09791201  0.09416234]
 [-0.09985273  0.10059027]
 [-0.09679924  0.1042345 ]
 [-0.10237681  0.1082252 ]
 [-0.10319786  0.10353966]
 [-0.09603477  0.09769294]
 [-0.09605227  0.09281462]
 [-0.09926391  0.09983173]
 [-0.09873319  0.10269979]
 [-0.10020103  0.10322214]
 [-0.09270011  0.09178396]
 

2017-09-27 13:39:58.877709: precision @ 1 = 0.546
2017-09-27 13:39:58.877817: precision @ 5 = 1.000
INFO:tensorflow:Restoring parameters from /tmp/kaist_train_simple/model.ckpt-1000
logit_val, label_val, predictions [[-0.09835029  0.10293818]
 [-0.09926391  0.09983173]
 [-0.10032994  0.09606221]
 [-0.09550121  0.09799665]
 [-0.11041923  0.11501108]
 [-0.09968746  0.10105972]
 [-0.09438189  0.09358783]
 [-0.10034698  0.10463602]
 [-0.09746464  0.09859048]
 [-0.09018295  0.09193168]
 [-0.10650436  0.10743897]
 [-0.09675451  0.09822359]
 [-0.09634648  0.1018796 ]
 [-0.10003662  0.09950507]
 [-0.09909277  0.10100006]
 [-0.10441469  0.10515486]
 [-0.10695099  0.1081838 ]
 [-0.09832387  0.09780813]
 [-0.0976854   0.09961041]
 [-0.09863178  0.10117306]
 [-0.10145406  0.10170314]
 [-0.09943024  0.09701587]
 [-0.09719494  0.09912039]
 [-0.10355097  0.10751959]
 [-0.09519817  0.09447741]
 [-0.09888143  0.09984361]
 [-0.10088396  0.10382837]
 [-0.10000567  0.1063305 ]
 [-0.10044679  0.10319692]
 

2017-09-27 13:50:10.074414: precision @ 1 = 0.557
2017-09-27 13:50:10.074523: precision @ 5 = 1.000
INFO:tensorflow:Restoring parameters from /tmp/kaist_train_simple/model.ckpt-1000
logit_val, label_val, predictions [[-0.09895741  0.10031177]
 [-0.10514235  0.10761438]
 [-0.09827976  0.09097421]
 [-0.09916532  0.10077526]
 [-0.10045563  0.10738377]
 [-0.09921525  0.09740572]
 [-0.09687492  0.09515904]
 [-0.10282668  0.1037491 ]
 [-0.10022486  0.10542395]
 [-0.09671056  0.09834373]
 [-0.10069395  0.10384307]
 [-0.09710942  0.10399048]
 [-0.09958863  0.10738785]
 [-0.09471844  0.09439403]
 [-0.10223903  0.10346854]
 [-0.10212077  0.10097866]
 [-0.10631655  0.10760048]
 [-0.09960306  0.09796576]
 [-0.10200801  0.10980617]
 [-0.10694326  0.10594182]
 [-0.09620001  0.09467126]
 [-0.09527844  0.09477471]
 [-0.10391012  0.1054167 ]
 [-0.10384905  0.10499977]
 [-0.10136542  0.10320172]
 [-0.09753095  0.09520533]
 [-0.10780238  0.10937557]
 [-0.0940586   0.10089877]
 [-0.1007569   0.10037088]
 

...

2017-09-27 15:32:01.891914: precision @ 1 = 0.554
2017-09-27 15:32:01.892026: precision @ 5 = 1.000
INFO:tensorflow:Restoring parameters from /tmp/kaist_train_simple/model.ckpt-1000
logit_val, label_val, predictions [[-0.09713333  0.09814633]
 [-0.09481335  0.09624939]
 [-0.09790913  0.09780657]
 [-0.10241945  0.10454032]
 [-0.09297831  0.09311433]
 [-0.09960548  0.0999644 ]
 [-0.10690778  0.1057215 ]
 [-0.09857982  0.09871796]
 [-0.09949329  0.10437413]
 [-0.10283574  0.10854704]
 [-0.09745362  0.09990739]
 [-0.10000567  0.1063305 ]
 [-0.10126932  0.10638914]
 [-0.09970235  0.09556   ]
 [-0.09651048  0.09599208]
 [-0.10315935  0.10423648]
 [-0.09741863  0.09663822]
 [-0.10240763  0.09882114]
 [-0.09671297  0.10095292]
 [-0.10326631  0.11347428]
 [-0.09579024  0.0965519 ]
 [-0.10425525  0.10548466]
 [-0.1000553   0.09810589]
 [-0.09551866  0.09931183]
 [-0.10139599  0.10198463]
 [-0.09219867  0.09370423]
 [-0.09557527  0.0963947 ]
 [-0.09623482  0.10052055]
 [-0.09678005  0.09574588]
 

2017-09-27 15:42:13.234790: precision @ 1 = 0.555
2017-09-27 15:42:13.234902: precision @ 5 = 1.000
INFO:tensorflow:Restoring parameters from /tmp/kaist_train_simple/model.ckpt-1000
logit_val, label_val, predictions [[-0.10117448  0.09722   ]
 [-0.0951224   0.09727925]
 [-0.09583675  0.09869915]
 [-0.10242565  0.10347666]
 [-0.09683494  0.09661875]
 [-0.10189401  0.10299788]
 [-0.09447293  0.09832081]
 [-0.10297618  0.10061257]
 [-0.09806495  0.099613  ]
 [-0.09665713  0.0939266 ]
 [-0.09831151  0.10153857]
 [-0.10335484  0.09851049]
 [-0.10013723  0.10446729]
 [-0.09575373  0.09953099]
 [-0.10657471  0.10949728]
 [-0.09684169  0.09425008]
 [-0.10022987  0.10183272]
 [-0.09791014  0.09686095]
 [-0.10093042  0.10098937]
 [-0.10024197  0.10328601]
 [-0.10124192  0.10627978]
 [-0.09630909  0.09714857]
 [-0.10020103  0.10322214]
 [-0.10090407  0.10238829]
 [-0.10237578  0.10572048]
 [-0.09376776  0.09506842]
 [-0.09797149  0.09584662]
 [-0.09778221  0.10040022]
 [-0.09745821  0.09997851]
 

2017-09-27 15:52:24.798442: precision @ 1 = 0.535
2017-09-27 15:52:24.798546: precision @ 5 = 1.000
INFO:tensorflow:Restoring parameters from /tmp/kaist_train_simple/model.ckpt-1000
logit_val, label_val, predictions [[-0.0902603   0.08841161]
 [-0.10277222  0.10561603]
 [-0.09985772  0.10504792]
 [-0.09791014  0.09686095]
 [-0.09912124  0.09889361]
 [-0.09524336  0.09604613]
 [-0.09655638  0.10074793]
 [-0.10151695  0.10337491]
 [-0.09791201  0.09416234]
 [-0.10432601  0.10642208]
 [-0.10078756  0.10020699]
 [-0.09608387  0.09793798]
 [-0.09452902  0.09303474]
 [-0.09676368  0.09780363]
 [-0.10193072  0.09817012]
 [-0.1000331   0.10205105]
 [-0.09919768  0.10099652]
 [-0.11327693  0.11444357]
 [-0.09623454  0.09874371]
 [-0.09966228  0.09950186]
 [-0.09320898  0.096076  ]
 [-0.09921737  0.1031553 ]
 [-0.10360369  0.10619591]
 [-0.09519115  0.09517152]
 [-0.09575909  0.10031596]
 [-0.10376006  0.10194606]
 [-0.09837071  0.1013917 ]
 [-0.09409969  0.09527558]
 [-0.09835105  0.10081986]
 

2017-09-27 16:02:35.957448: precision @ 1 = 0.560
2017-09-27 16:02:35.957582: precision @ 5 = 1.000
INFO:tensorflow:Restoring parameters from /tmp/kaist_train_simple/model.ckpt-1000
logit_val, label_val, predictions [[-0.1013951   0.09889385]
 [-0.09636014  0.10316096]
 [-0.10151695  0.10337491]
 [-0.10224136  0.10745061]
 [-0.09871256  0.09696935]
 [-0.09600969  0.09777804]
 [-0.09878035  0.10407751]
 [-0.09751605  0.09900519]
 [-0.10115848  0.10268542]
 [-0.09455852  0.09890916]
 [-0.09929913  0.10288299]
 [-0.10562102  0.10039842]
 [-0.09493053  0.10026048]
 [-0.09725347  0.09815404]
 [-0.10274083  0.10377106]
 [-0.09789471  0.09515135]
 [-0.0936583   0.09852359]
 [-0.10917305  0.10795565]
 [-0.10142948  0.10488902]
 [-0.09728725  0.09834158]
 [-0.10265739  0.10220004]
 [-0.09616896  0.10019352]
 [-0.09786401  0.10095093]
 [-0.09808986  0.10471   ]
 [-0.09550537  0.09447129]
 [-0.10080518  0.09746291]
 [-0.09821672  0.10088967]
 [-0.095332    0.09507683]
 [-0.10200801  0.10980617]
 

2017-09-27 16:12:46.887490: precision @ 1 = 0.543
2017-09-27 16:12:46.887600: precision @ 5 = 1.000
INFO:tensorflow:Restoring parameters from /tmp/kaist_train_simple/model.ckpt-1000
logit_val, label_val, predictions [[-0.10204405  0.10407095]
 [-0.09409969  0.09527558]
 [-0.10464181  0.10009804]
 [-0.09325464  0.09735069]
 [-0.09900367  0.10000683]
 [-0.10199574  0.09956463]
 [-0.10023307  0.10301521]
 [-0.09201273  0.09849289]
 [-0.09959351  0.10400017]
 [-0.10911609  0.10761428]
 [-0.10344312  0.09852647]
 [-0.09904127  0.10036877]
 [-0.09201332  0.09325035]
 [-0.09738171  0.09934571]
 [-0.09725347  0.09815404]
 [-0.09588362  0.10008615]
 [-0.09458789  0.10088887]
 [-0.09398006  0.10036527]
 [-0.09371903  0.09992128]
 [-0.10240763  0.09882114]
 [-0.09758732  0.09986456]
 [-0.0997199   0.10279676]
 [-0.09982367  0.10120931]
 [-0.09701023  0.09848103]
 [-0.09999195  0.10041412]
 [-0.09875557  0.09942853]
 [-0.09660999  0.10086673]
 [-0.09888633  0.09484717]
 [-0.09988434  0.10694811]
 